In [29]:
import numpy as np
from BPmodule.BPGen import CreateData
import pandas as pd
from numba import njit, objmode, prange
from numba.typed import Dict
from numba.core import types
import time
from BPnumba.GeneticOperators import CreateHeuristicPob, Ind, InstancePob

from numba.typed import List as NumbaList
import matplotlib.pyplot as plt
from typing import List
from numpy import savetxt
from BPnumba.Bench import Test
from BPnumba.DFFA import createDFFA
from BPnumba.EDFFA import createEDFFA
from BPnumba.DABC import createDABC
from BPnumba.EDABC import createEDABC 
from BPnumba.AGH import createAG,NAG  

In [30]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)

createText = lambda x,y : savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=" ",
    fmt='%g'
)

numTest = 20
num = 5
agL = zerovec(num, 2*numTest)
dabcL = zerovec(num, 2*numTest)
ffaL = zerovec(num, 2*numTest)

pr_selec = 0.85
pr_cross = 0.75
maxItr=1000
pop = 50

In [31]:
"""contenedor,datos = CreateData(algorithm=1,problem=1)

start = time.time()
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True)
Test(datos.copy(),contenedor,aga, maxItr, agL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin ag 10 ",minutes,":",seconds)
start = time.time()
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1)
Test(datos.copy(),contenedor,aga, maxItr, agL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin ag 20 ",minutes,":",seconds)

start = time.time()
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,mutType=1)
Test(datos.copy(),contenedor,aga, maxItr, agL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin ag 11 ",minutes,":",seconds)
start = time.time()
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1,mutType=1)
Test(datos.copy(),contenedor,aga, maxItr, agL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin ag 21 ",minutes,":",seconds)


start = time.time()
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,mutType=2)
Test(datos.copy(),contenedor,aga, maxItr, agL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin ag 12 ",minutes,":",seconds)
start = time.time()
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1,mutType=2)
Test(datos.copy(),contenedor,aga, maxItr, agL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin ag 22 ",minutes,":",seconds)

#'''''''

start = time.time()
dabc = createDABC()
Test(datos.copy(),contenedor,dabc, maxItr, dabcL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin dabc 1 ",minutes,":",seconds)
start = time.time()
dabc = createDABC(1)
Test(datos.copy(),contenedor,dabc, maxItr, dabcL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin dabc 2 ",minutes,":",seconds)

start = time.time()
dabc = createEDABC()
Test(datos.copy(),contenedor,dabc, maxItr, dabcL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edabc 1 ",minutes,":",seconds)
start = time.time()
dabc = createEDABC(1)
Test(datos.copy(),contenedor,dabc, maxItr, dabcL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edabc 2 ",minutes,":",seconds)

start = time.time()
effa = createDFFA()
Test(datos.copy(),contenedor,effa, maxItr, ffaL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin dffa 1 ",minutes,":",seconds)
start = time.time()
effa = createDFFA(1)
Test(datos.copy(),contenedor,effa, maxItr, ffaL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin dffa 2 ",minutes,":",seconds)

start = time.time()
effa = createEDFFA()
Test(datos.copy(),contenedor,effa, maxItr, ffaL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edffa 1 ",minutes,":",seconds)
start = time.time()
effa = createEDFFA(1)
Test(datos.copy(),contenedor,effa, maxItr, ffaL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edffa 2 ",minutes,":",seconds)

start = time.time()
effa = createEDFFA(mutType=1)
Test(datos.copy(),contenedor,effa, maxItr, ffaL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edffa 11",minutes,":",seconds)
start = time.time()
effa = createEDFFA(heuristic=1,mutType=1)
Test(datos.copy(),contenedor,effa, maxItr, ffaL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edffa 21 ",minutes,":",seconds)


start = time.time()
effa = createEDFFA(mutType=2)
Test(datos.copy(),contenedor,effa, maxItr, ffaL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edffa 12 ",minutes,":",seconds)
start = time.time()
effa = createEDFFA(heuristic=1,mutType=2)
Test(datos.copy(),contenedor,effa, maxItr, ffaL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edffa 22 ",minutes,":",seconds)
"""

'contenedor,datos = CreateData(algorithm=1,problem=1)\n\nstart = time.time()\naga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True)\nTest(datos.copy(),contenedor,aga, maxItr, agL)\nminutes = round((time.time()-start)/ 60)\nseconds = int((time.time()-start) % 60)\nprint("fin ag 10 ",minutes,":",seconds)\nstart = time.time()\naga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1)\nTest(datos.copy(),contenedor,aga, maxItr, agL,1)\nminutes = round((time.time()-start)/ 60)\nseconds = int((time.time()-start) % 60)\nprint("fin ag 20 ",minutes,":",seconds)\n\nstart = time.time()\naga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,mutType=1)\nTest(datos.copy(),contenedor,aga, maxItr, agL)\nminutes = round((time.time()-start)/ 60)\nseconds = int((time.time()-start) % 60)\nprint("fin ag 11 ",minutes,":",seconds)\nstart = time.time()\naga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1,mutType=1)\nTest(datos.copy(),contenedor,aga, maxItr, agL,1)\nminutes = round((time.tim

In [33]:
@njit(parallel=True)
def Bench(boxes:List[List[List[int]]],bin:List[int],maxItr: int, lst: List[List[float]], lst2:List[List[float]],heuristic:int=0):
    for i in prange(20):
        alg = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=heuristic)
        alg.Train(maxItr,  InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, heuristic), NumbaList(boxes[i]), bin)
        if heuristic == 0:
            lst[i] =alg.BestInd.fi
            lst2.append(alg.bestfi)
        else:
            lst2.append(alg.bestfi)
            lst[20+i] = alg.BestInd.fi


In [39]:
algoritms = [1,2]
problems = [3]
for problem in problems:
    for algoritm in algoritms:
        agL = zerovec(1, 2*numTest)
        agL2:List[List[float]] = NumbaList([NumbaList(np.array([0.0],dtype=np.float64))])
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Bench(datos.copy(),contenedor, maxItr, agL,agL2)
        createText("P"+str(problem)+"A"+str(algoritm)+"/HAG",agL)
        Bench(datos.copy(),contenedor, maxItr, agL,agL2,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/HAG",agL)
        agL2.pop(0)
        createText("P"+str(problem)+"A"+str(algoritm)+"/_HAG",agL2)

In [ ]:
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,mutType=1)
aga2 = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1,mutType=1)
for problem in problems:
    for algoritm in algoritms:
        agL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor,aga, maxItr, agL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1HAG",agL)
        Test(datos.copy(),contenedor,aga2, maxItr, agL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1HAG",agL)

aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,mutType=2)
aga2 = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1,mutType=2)
for problem in problems:
    for algoritm in algoritms:
        agL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor,aga, maxItr, agL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c2HAG",agL)
        Test(datos.copy(),contenedor,aga2, maxItr, agL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c2HAG",agL)

In [ ]:
effa = createEDFFA()
effa2 = createEDFFA(heuristic= 1)
for problem in problems:
    for algoritm in algoritms:
        ffaL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDFFA",ffaL)
        
effa = createEDFFA(mutType=1)
effa2 = createEDFFA(heuristic= 1,mutType=1)
for problem in problems:
    for algoritm in algoritms:
        ffaL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1EDFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1EDFFA",ffaL)

effa = createEDFFA(mutType=2)
effa2 = createEDFFA(heuristic= 1,mutType=2)
for problem in problems:
    for algoritm in algoritms:
        ffaL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c2EDFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c2EDFFA",ffaL)

In [ ]:
dabc = createDABC()
dabc2 = createDABC(1)

for problem in problems:
    for algoritm in algoritms:
        dabcL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, dabc,maxItr, dabcL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DABC",dabcL)
        Test(datos.copy(),contenedor, dabc2,maxItr, dabcL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DABC",dabcL)
        
dabc = createEDABC()
dabc2 = createEDABC(1)
for problem in problems:
    for algoritm in algoritms:
        dabcL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, dabc,maxItr, dabcL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDABC",dabcL)
        Test(datos.copy(),contenedor, dabc2,maxItr, dabcL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDABC",dabcL)

In [ ]:
effa = createDFFA()
effa2 = createDFFA(heuristic= 1)

for problem in problems:
    for algoritm in algoritms:
        ffaL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DFFA",ffaL)

In [ ]:
algoritms = [1,2]
problems = [3]

In [ ]:
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True)
aga2 = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1)
for problem in problems:
    for algoritm in algoritms:
        agL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor,aga, maxItr, agL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/HAG",agL)
        Test(datos.copy(),contenedor,aga2, maxItr, agL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/HAG",agL)

aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,mutType=1)
aga2 = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1,mutType=1)
for problem in problems:
    for algoritm in algoritms:
        agL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor,aga, maxItr, agL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1HAG",agL)
        Test(datos.copy(),contenedor,aga2, maxItr, agL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1HAG",agL)

aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,mutType=2)
aga2 = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1,mutType=2)
for problem in problems:
    for algoritm in algoritms:
        agL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor,aga, maxItr, agL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c2HAG",agL)
        Test(datos.copy(),contenedor,aga2, maxItr, agL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c2HAG",agL)

effa = createEDFFA()
effa2 = createEDFFA(heuristic= 1)
for problem in problems:
    for algoritm in algoritms:
        ffaL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDFFA",ffaL)

effa = createEDFFA(mutType=1)
effa2 = createEDFFA(heuristic= 1,mutType=1)
for problem in problems:
    for algoritm in algoritms:
        ffaL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1EDFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1EDFFA",ffaL)

effa = createEDFFA(mutType=2)
effa2 = createEDFFA(heuristic= 1,mutType=2)
for problem in problems:
    for algoritm in algoritms:
        ffaL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c2EDFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c2EDFFA",ffaL)
dabc = createDABC()
dabc2 = createDABC(1)

for problem in problems:
    for algoritm in algoritms:
        dabcL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, dabc,maxItr, dabcL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DABC",dabcL)
        Test(datos.copy(),contenedor, dabc2,maxItr, dabcL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DABC",dabcL)
        
dabc = createEDABC()
dabc2 = createEDABC(1)
for problem in problems:
    for algoritm in algoritms:
        dabcL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, dabc,maxItr, dabcL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDABC",dabcL)
        Test(datos.copy(),contenedor, dabc2,maxItr, dabcL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDABC",dabcL)
effa = createDFFA()
effa2 = createDFFA(heuristic= 1)
for problem in problems:
    for algoritm in algoritms:
        ffaL = zerovec(num, 2*numTest)
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DFFA",ffaL)